In [1]:
import Pkg

In [9]:
pwd()

"/Users/fos08b/repos/NREL-Sienna"

In [10]:
Pkg.activate("./PowerSystemsInvestments.jl/test/Project.toml")

  Activating project at `~/repos/NREL-Sienna/PowerSystemsInvestments.jl/test`


In [11]:
Pkg.status()

Status `~/repos/NREL-Sienna/PowerSystemsInvestments.jl/test/Project.toml`
  [4c88cf16] Aqua v0.8.14
  [336ed68f] CSV v0.10.15
  [a93c6f00] DataFrames v1.7.0
  [87dc4568] HiGHS v1.19.0
  [2cd47ed4] InfrastructureSystems v2.6.0
  [4076af6c] JuMP v1.28.0
  [c36e90e8] PowerModels v0.21.5
  [f00506e0] PowerSystemCaseBuilder v1.3.11
  [bcd98974] PowerSystems v4.6.2 `~/.julia/dev/PowerSystems`
  [bed98974] PowerSystemsInvestments v0.1.0 `..`
  [bed98974] PowerSystemsInvestmentsPortfolios v0.1.0 `~/.julia/dev/PowerSystemsInvestmentsPortfolios`
  [295af30f] Revise v3.9.0
  [9e3dc215] TimeSeries v0.24.2
  [ade2ca70] Dates v1.11.0
  [8dfed614] Test v1.11.0


In [50]:
using Test
import InfrastructureSystems
import InfrastructureSystems.Optimization
using JuMP
using Logging
using PowerSystemsInvestments
using PowerSystemsInvestmentsPortfolios
using PowerSystems
using DataFrames
using HiGHS

using Test
import InfrastructureSystems
import InfrastructureSystems.Optimization
using JuMP
using Logging
using PowerSystemsInvestments
using PowerSystemsInvestmentsPortfolios
using PowerSystems
using DataFrames
using HiGHS
using TimeSeries
using Dates
using Statistics

const IS = InfrastructureSystems
const PSIP = PowerSystemsInvestmentsPortfolios
const PSIN = PowerSystemsInvestments
const PSY = PowerSystems

PowerSystems

In [31]:
const IS = InfrastructureSystems
const PSIP = PowerSystemsInvestmentsPortfolios
const PSIN = PowerSystemsInvestments
const PSY = PowerSystems

PowerSystems

In [32]:
using PowerSystemCaseBuilder

In [33]:
discount_rate = 0.07 # 7% 
inflation_rate = 0.05 # 5%
interest_rate = 0.04 # 4%

0.04

In [34]:
sys = build_system(PSITestSystems, "c_sys5_re")

[ Info: Deserializing with InMemoryTimeSeriesStorage is currently not supported. Using HDF
[ Info: Loaded time series from storage file existing=/Users/fos08b/.julia/packages/PowerSystemCaseBuilder/dJGb8/data/serialized_system/4e67b70ea6977dbe21c7731d72cdc1494adf072a7f3f08d921db740cf264ce79/c_sys5_re_time_series_storage.h5 new=/var/folders/_t/4yx83vws0ls_k18lgqrg8wrc0000gn/T/jl_siqaCM compression=CompressionSettings(false, CompressionTypes.DEFLATE = 1, 3, true)


Property,Value
Name,
Description,
System Units Base,SYSTEM_BASE
Base Power,100.0
Base Frequency,60.0
Num Components,28
Type,Count
ACBus,5
Arc,6
Line,6


In [35]:
set_units_base_system!(sys, "NATURAL_UNITS")

[ Info: Unit System changed to UnitSystem.NATURAL_UNITS = 2


In [36]:
# Zones
z1 = Zone(name="Zone_1", id=1)
z2 = Zone(name="Zone_2", id=2)

Zone: Zone_2:
   name: Zone_2
   id: 2
   ext: Dict{Any, Any}()
   has_time_series: false

In [37]:
# Time Series:
tstamp_2030_ops = collect(
        DateTime("1/1/2030 0:00:00", "d/m/y H:M:S"):Hour(1):
        DateTime("1/1/2030 23:00:00", "d/m/y H:M:S")
)

24-element Vector{DateTime}:
 2030-01-01T00:00:00
 2030-01-01T01:00:00
 2030-01-01T02:00:00
 2030-01-01T03:00:00
 2030-01-01T04:00:00
 2030-01-01T05:00:00
 2030-01-01T06:00:00
 2030-01-01T07:00:00
 2030-01-01T08:00:00
 2030-01-01T09:00:00
 2030-01-01T10:00:00
 2030-01-01T11:00:00
 2030-01-01T12:00:00
 2030-01-01T13:00:00
 2030-01-01T14:00:00
 2030-01-01T15:00:00
 2030-01-01T16:00:00
 2030-01-01T17:00:00
 2030-01-01T18:00:00
 2030-01-01T19:00:00
 2030-01-01T20:00:00
 2030-01-01T21:00:00
 2030-01-01T22:00:00
 2030-01-01T23:00:00

In [38]:
tstamp_2035_ops = collect(
        DateTime("1/1/2035 0:00:00", "d/m/y H:M:S"):Hour(1):
        DateTime("1/1/2035 23:00:00", "d/m/y H:M:S")
)

24-element Vector{DateTime}:
 2035-01-01T00:00:00
 2035-01-01T01:00:00
 2035-01-01T02:00:00
 2035-01-01T03:00:00
 2035-01-01T04:00:00
 2035-01-01T05:00:00
 2035-01-01T06:00:00
 2035-01-01T07:00:00
 2035-01-01T08:00:00
 2035-01-01T09:00:00
 2035-01-01T10:00:00
 2035-01-01T11:00:00
 2035-01-01T12:00:00
 2035-01-01T13:00:00
 2035-01-01T14:00:00
 2035-01-01T15:00:00
 2035-01-01T16:00:00
 2035-01-01T17:00:00
 2035-01-01T18:00:00
 2035-01-01T19:00:00
 2035-01-01T20:00:00
 2035-01-01T21:00:00
 2035-01-01T22:00:00
 2035-01-01T23:00:00

In [39]:
tstamp_inv = [
    DateTime("1/1/2030  0:00:00", "d/m/y  H:M:S"),
    DateTime("1/1/2035  0:00:00", "d/m/y  H:M:S"),
]

2-element Vector{DateTime}:
 2030-01-01T00:00:00
 2035-01-01T00:00:00

In [40]:
?TechnologyFinancialData(;)

```
mutable struct TechnologyFinancialData <: FinancialData
    interest_rate::Float64
    capital_recovery_period::Int64
    technology_base_year::Int64
end
```

# Arguments

  * `interest_rate::Float64`: (default: `0.0`) Interest rate
  * `capital_recovery_period::Int64`: Capital recovery period (in years) used for determining overnight capital costs from annualized investment costs
  * `technology_base_year::Int64`: Year for which all dollar values for this technology are reported. Will be converted to dollars in the portfolio base year.


In [84]:
tech_financials() = TechnologyFinancialData(; interest_rate=0.04, capital_recovery_period=30, technology_base_year=2025)

tech_financials (generic function with 1 method)

In [73]:
thermals = collect(get_components(ThermalStandard, sys)); thermals[1]

ThermalStandard: Solitude:
   name: Solitude
   available: true
   status: true
   bus: ACBus: nodeC
   active_power: 520.0
   reactive_power: 100.0
   rating: 520.0
   active_power_limits: (min = 0.0, max = 520.0)
   reactive_power_limits: (min = -390.0, max = 390.0)
   ramp_limits: (up = 6.24, down = 6.24)
   operation_cost: ThermalGenerationCost composed of variable: CostCurve{LinearCurve}
   base_power: 100.0
   time_limits: (up = 3.0, down = 2.0)
   must_run: false
   prime_mover_type: PrimeMovers.ST = 20
   fuel: ThermalFuels.COAL = 1
   services: 0-element Vector{Service}
   time_at_status: 10000.0
   dynamic_injector: nothing
   ext: Dict{String, Any}()
   InfrastructureSystems.SystemUnitsSettings:
      base_value: 100.0
      unit_system: UnitSystem.NATURAL_UNITS = 2
   has_supplemental_attributes: false
   has_time_series: false

In [74]:
var_cost = PSY.get_variable.((get_operation_cost.((thermals))))
op_cost = PSY.get_proportional_term.(get_value_curve.(var_cost))

cheap_th_ixs = 2:4 # cheap thermal
exp_th_ixs = [1, 5] # expensive!
cheap_th_var_cost = mean(op_cost[cheap_th_ixs])
exp_th_var_cost = mean(op_cost[exp_th_ixs])
cheap_th_names = PSY.get_name.(thermals[cheap_th_ixs])
exp_th_names = PSY.get_name.(thermals[exp_th_ixs])

2-element Vector{String}:
 "Solitude"
 "Sundance"

In [75]:
cheap_th_names

3-element Vector{String}:
 "Park City"
 "Alta"
 "Brighton"

In [76]:
# Year 2030:
coal_igcc_capex = 6937.377 # $/kW
coal_new_capex = 3823.56 # $/kW

# Year 2035:
coal_igcc_capex_2035 = 6869.263 # $/kW
coal_new_capex_2035 = 3664.307 # $/kW

3664.307

In [77]:
ts_th_cheap_inv_capex = SingleTimeSeries(
    "inv_capex",
    TimeArray(tstamp_inv, [1.0, coal_igcc_capex / coal_igcc_capex_2035]),
)
ts_th_exp_inv_capex = SingleTimeSeries(
    "inv_capex",
    TimeArray(tstamp_inv, [1.0, coal_new_capex / coal_new_capex_2035]),
)

SingleTimeSeries("inv_capex", 2×1 TimeArray{Float64, 1, DateTime, Vector{Float64}} 2030-01-01T00:00:00 to 2035-01-01T00:00:00, Millisecond(157766400000), nothing, InfrastructureSystems.InfrastructureSystemsInternal(Base.UUID("2730197e-0c56-489b-bc32-f09a958445d9"), nothing, nothing, nothing))

In [78]:
ts_th_exp_inv_capex.data

2×1 TimeArray{Float64, 1, DateTime, Vector{Float64}} 2030-01-01T00:00:00 to 2035-01-01T00:00:00
┌─────────────────────┬─────────┐
│                     │ A       │
├─────────────────────┼─────────┤
│ 2030-01-01T00:00:00 │     1.0 │
│ 2035-01-01T00:00:00 │ 1.04346 │
└─────────────────────┴─────────┘

In [81]:
t_th = SupplyTechnology{ThermalStandard}(;
    prime_mover_type=PrimeMovers.ST,
    capital_costs=LinearCurve(coal_igcc_capex * 1000.0),
    id=1,
    available=true,
    name="cheap_thermal",
    fuel=[ThermalFuels.COAL],
    power_systems_type="ThermalStandard",
    operation_costs=ThermalGenerationCost(
        variable=CostCurve(LinearCurve(cheap_th_var_cost)),
        fixed=0.0,
        start_up=0.0,
        shut_down=0.0,
    ),#LinearCurve(0.0),
    capacity_limits=(0.0, 1e8),
    outage_factor=1.0,
    region=[z1],
    unit_size=250.0,
    financial_data=tech_financials(),
)

SupplyTechnology: cheap_thermal:
   outage_factor: 1.0
   prime_mover_type: PrimeMovers.ST = 20
   capital_costs: LinearCurve(6.937377e6, 0.0)
   lifetime: 100
   available: true
   co2: Dict{ThermalFuels, Float64}()
   name: cheap_thermal
   id: 1
   cofire_start_limits: Dict{ThermalFuels, @NamedTuple{min::Float64, max::Float64}}()
   financial_data: TechnologyFinancialData
   start_fuel_mmbtu_per_mw: 0.0
   operation_costs: ThermalGenerationCost composed of variable: CostCurve{LinearCurve}
   fuel: ThermalFuels[ThermalFuels.COAL = 1]
   power_systems_type: ThermalStandard
   cofire_level_limits: Dict{ThermalFuels, @NamedTuple{min::Float64, max::Float64}}()
   nothing
   ext: Dict{Any, Any}()
   region: 1-element Vector{RegionTopology}
   min_generation_fraction: 0.0
   time_limits: (up = 1.0, down = 1.0)
   unit_size: 250.0
   ramp_limits: (up = 1.0, down = 1.0)
   capacity_limits: (min = 0.0, max = 1.0e8)
   has_supplemental_attributes: false
   has_time_series: false

In [85]:
t_th_exp = SupplyTechnology{ThermalStandard}(;
    prime_mover_type=PrimeMovers.ST,
    capital_costs=LinearCurve(coal_new_capex * 1000.0),
    id=2,
    available=true,
    name="expensive_thermal",
    fuel=[ThermalFuels.COAL],
    power_systems_type="ThermalStandard",
    operation_costs=ThermalGenerationCost(
        variable=CostCurve(LinearCurve(exp_th_var_cost)),
        fixed=0.0,
        start_up=0.0,
        shut_down=0.0,
    ),
    capacity_limits=(0.0, 1e8),
    outage_factor=1.0,
    region=[z1],
    unit_size=75.0,
    financial_data=tech_financials(),
)

SupplyTechnology: expensive_thermal:
   outage_factor: 1.0
   prime_mover_type: PrimeMovers.ST = 20
   capital_costs: LinearCurve(3.82356e6, 0.0)
   lifetime: 100
   available: true
   co2: Dict{ThermalFuels, Float64}()
   name: expensive_thermal
   id: 2
   cofire_start_limits: Dict{ThermalFuels, @NamedTuple{min::Float64, max::Float64}}()
   financial_data: TechnologyFinancialData
   start_fuel_mmbtu_per_mw: 0.0
   operation_costs: ThermalGenerationCost composed of variable: CostCurve{LinearCurve}
   fuel: ThermalFuels[ThermalFuels.COAL = 1]
   power_systems_type: ThermalStandard
   cofire_level_limits: Dict{ThermalFuels, @NamedTuple{min::Float64, max::Float64}}()
   nothing
   ext: Dict{Any, Any}()
   region: 1-element Vector{RegionTopology}
   min_generation_fraction: 0.0
   time_limits: (up = 1.0, down = 1.0)
   unit_size: 75.0
   ramp_limits: (up = 1.0, down = 1.0)
   capacity_limits: (min = 0.0, max = 1.0e8)
   has_supplemental_attributes: false
   has_time_series: false

In [56]:
?Portfolio

search: Portfolio



Construct an empty `Portfolio`. Useful for building a Portfolio from scratch.

---

Construct an empty `Portfolio` specifying aggregation. Useful for building a Portfolio from scratch.

---

Construct an empty `Portfolio` specifying base_system. Useful for building a Portfolio from scratch.

---

Construct an empty `Portfolio` specifying financial data. Useful for building a Portfolio from scratch.

---

Construct an empty `Portfolio` specifying financial data. Useful for building a Portfolio from scratch and used in the database parser.

---

Constructs a Portfolio from a file path ending with .json

If the file is JSON, then `assign_new_uuids = true` will generate new UUIDs for the system and all components.


In [57]:
p_5bus = Portfolio(
    sys;
    financial_data = PortfolioFinancialData(
        2025, # base_year
        discount_rate,
        inflation_rate,
        interest_rate,
    )
)

Property,Value
Name,
Description,
Aggregation,Area


In [58]:
PSIP.set_name!(p_5bus, "Our Test Portfolio System")

"Our Test Portfolio System"

In [59]:
p_5bus

Property,Value
Name,Our Test Portfolio System
Description,
Aggregation,Area


In [60]:
PSIP.add_region!(p_5bus, z1)

┌ Warning: struct Zone does not exist in validation configuration file, validation skipped
└ @ InfrastructureSystems ~/.julia/packages/InfrastructureSystems/LEg3t/src/validation.jl:51


In [61]:
PSIP.add_region!(p_5bus, z2)

┌ Warning: struct Zone does not exist in validation configuration file, validation skipped
└ @ InfrastructureSystems ~/.julia/packages/InfrastructureSystems/LEg3t/src/validation.jl:51


In [82]:
PSIP.add_technology!(p_5bus, t_th)

In [86]:
PSIP.add_technology!(p_5bus, t_th_exp)

In [87]:
p_5bus

Property,Value
Name,Our Test Portfolio System
Description,
Aggregation,Area
Type,Count
SupplyTechnology,2


In [88]:
?DemandSideTechnology

search: DemandSideTechnology DemandTechnology ResourceTechnology add_technology!



```
mutable struct DemandSideTechnology{T <: PSY.StaticInjection} <: DemandTechnology
    price_per_unit::PSY.ValueCurve
    available::Bool
    name::String
    shift_variable_cost::PSY.ValueCurve
    curtailment_cost::PSY.ValueCurve
    id::Int64
    technology_efficiency::Float64
    max_demand_advance::Float64
    demand_energy_efficiency::Float64
    max_demand_curtailment::Float64
    max_demand_delay::Float64
    power_systems_type::String
    internal::InfrastructureSystemsInternal
    ext::Dict
    region::Vector{RegionTopology}
    min_power::Float64
    peak_demand_mw::Float64
end
```

Represents demand side technologies such as electric vehicles or hydrogen electrolyzers.

# Arguments

  * `price_per_unit::PSY.ValueCurve`: (default: `LinearCurve(0.0)`) Price or value per unit of output. Ex: USD per ton of hydrogen for electrolyzers
  * `available::Bool`: Indicator of whether the component is connected and online (`true`) or disconnected, offline, or down (`false`)
  * `name::String`: The technology name
  * `shift_variable_cost::PSY.ValueCurve`: (default: `LinearCurve(0.0)`) Variable operation and maintenance costs associated with flexible demand deferral/advancement
  * `curtailment_cost::PSY.ValueCurve`: (default: `LinearCurve(0.0)`) Energy cost of curtailed demand, USD per Mwh
  * `id::Int64`: ID for demand side technology
  * `technology_efficiency::Float64`: (default: `0.0`) MWh of electricity per unit of output. Ex: MWh per ton of hydrogen for electrolyzers
  * `max_demand_advance::Float64`: (default: `0.0`) Maximum number of hours that demand can be scheduled in advance of the original schedule (hours).
  * `demand_energy_efficiency::Float64`: (default: `0.0`) Energy efficiency associated with time shifting demand. Represents energy losses due to time shifting
  * `max_demand_curtailment::Float64`: (default: `0.0`) Maximum fraction of demand that can be curtailed
  * `max_demand_delay::Float64`: (default: `0.0`) Maximum number of hours that demand can be deferred or delayed (hours).
  * `power_systems_type::String`: maps to a valid PowerSystems.jl for PCM modeling
  * `internal::InfrastructureSystemsInternal`: (default: `InfrastructureSystemsInternal()`) Internal field
  * `ext::Dict`: (default: `Dict()`) Option for providing additional data
  * `region::Vector{RegionTopology}`: (default: `Vector()`) Location where technology is operated
  * `min_power::Float64`: (default: `0.0`) Minimum operation of demandside unit as a fraction of peak demand
  * `peak_demand_mw::Float64`: (default: `0.0`) Peak demand value in MW


In [89]:
?DemandTechnology

search: DemandTechnology DemandSideTechnology StorageTechnology Technology



No documentation found for public symbol.

# Summary

```
abstract type DemandTechnology
```

# Subtypes

```
DemandRequirement{T<:StaticInjection}
DemandSideTechnology{T<:StaticInjection}
```

# Supertype Hierarchy

```
DemandTechnology <: Technology <: InfrastructureSystems.InfrastructureSystemsComponent <: InfrastructureSystems.InfrastructureSystemsType <: Any
```
